In [45]:
import pandas as pd
import re

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [8]:
data = pd.read_csv('/home/shuming-new/Downloads/reviews.csv', sep="|", header=0)
print(data)


           label                                               text
0       positive  Great CD: My lovely Pat has one of the GREAT v...
1       positive  One of the best game music soundtracks - for a...
2       negative  Batteries died within a year ...: I bought thi...
3       positive  works fine, but Maha Energy is better: Check o...
4       positive  Great for the non-audiophile: Reviewed quite a...
5       negative  DVD Player crapped out after one year: I also ...
6       negative  Incorrect Disc: I love the style of this, but ...
7       negative  DVD menu select problems: I cannot scroll thro...
8       positive  Unique Weird Orientalia from the 1930's: Exoti...
9       negative  Not an "ultimate guide": Firstly,I enjoyed the...
10      positive  Great book for travelling Europe: I currently ...
11      negative  Not!: If you want to listen to El Duke , then ...
12      negative  A complete Bust: This game requires quicktime ...
13      positive  TRULY MADE A DIFFERENCE!: I ha

In [24]:
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x).lower()))

In [25]:
print(data)

           label                                               text
0       positive  great cd my lovely pat has one of the great vo...
1       positive  one of the best game music soundtracks  for a ...
2       negative  batteries died within a year  i bought this ch...
3       positive  works fine but maha energy is better check out...
4       positive  great for the nonaudiophile reviewed quite a b...
5       negative  dvd player crapped out after one year i also b...
6       negative  incorrect disc i love the style of this but af...
7       negative  dvd menu select problems i cannot scroll throu...
8       positive  unique weird orientalia from the 1930s exotic ...
9       negative  not an ultimate guide firstlyi enjoyed the for...
10      positive  great book for travelling europe i currently l...
11      negative  not if you want to listen to el duke  then it ...
12      negative  a complete bust this game requires quicktime 5...
13      positive  truly made a difference i have

In [27]:
tokenizer = Tokenizer(num_words=2500, lower=True,split=' ')

In [28]:
tokenizer.fit_on_texts(data['text'].values)

In [33]:
X = tokenizer.texts_to_sequences(data['text'].values)

In [37]:
print(X[0])
print(X[1])

[28, 95, 21, 1565, 43, 24, 6, 1, 28, 1926, 6, 77, 1969, 3, 20, 995, 5, 8, 95, 11, 135, 2, 3, 122, 79, 7, 49, 106, 10, 4, 32, 1995, 7, 199, 44, 228, 90, 4, 124, 1995, 37, 31, 2446, 10, 1, 2340, 8, 95, 37, 161, 1078, 23, 2, 645, 37, 1508, 24, 6, 8, 9, 4, 95, 10, 21, 19, 176, 96, 119, 127, 7, 217, 9, 37, 846, 44, 3, 235, 8, 53, 610, 427, 589, 464, 138, 1819, 1350, 2196, 501, 24, 162, 70, 13, 12, 912]
[24, 6, 1, 87, 144, 120, 11, 4, 144, 3, 133, 64, 235, 1050, 1, 333, 12, 3, 20, 55, 516, 4, 246, 6, 1, 144, 1, 120, 3, 313, 641, 1, 1318, 5, 88, 13, 28, 22, 68, 1739, 44, 5, 273, 1, 1008, 2, 7, 2433, 24, 6, 21, 294, 559, 60, 9, 41, 950, 921, 6, 211, 2, 1639, 159, 177, 698, 2, 359, 524, 3, 347, 31, 22, 481, 14, 83, 109, 6, 177, 6, 159, 10, 21, 71, 276, 144, 3, 180, 1222, 12, 24, 6, 1, 159, 2409, 43, 1079, 1994, 5, 21, 895, 18, 109, 24, 1258, 40, 8, 1008, 9, 12, 33, 89, 706, 943, 10, 109, 6, 1, 159, 88, 3, 136, 17, 74, 30, 177, 1576, 734, 3, 39, 122, 1005, 1, 368, 158, 7]


In [38]:
X = pad_sequences(X)

In [39]:
print(X[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [42]:
print(len(X[0]))

852


In [49]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 852, 128)          320000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 852, 128)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 402       
Total params: 583,602
Trainable params: 583,602
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
Y = pd.get_dummies(data['label']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.2, random_state = 69)
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 2, verbose = 5)

Epoch 1/2
Epoch 2/2


In [54]:
score, acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Validation Accuracy: %.2f" % (acc))


Score: 0.19
Validation Accuracy: 0.93


In [55]:
model.save('amazon_review.h5')